<a href="https://colab.research.google.com/github/ahzaidy/Programs/blob/main/Copy_of_CPSC_5450_HW1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

# Load word files with encoding

def load_words(file_path, encoding="utf-8"):
    with open(file_path, 'r', encoding=encoding) as file:
        return [line.strip() for line in file.readlines()]

english_words = load_words('/content/drive/My Drive/english.txt')
german_words = load_words('/content/drive/My Drive/german.txt', encoding="latin1")
spanish_words = load_words('/content/drive/My Drive/spanish.txt')

# Filter only 5-letter words
def filter_five_letter_words(word_list):
    return [word for word in word_list if len(word) == 5]

english_words = filter_five_letter_words(english_words)
german_words = filter_five_letter_words(german_words)
spanish_words = filter_five_letter_words(spanish_words)

# Assign labels: English = 0, German = 1, Spanish = 2
english_labels = [0] * len(english_words)
german_labels = [1] * len(german_words)
spanish_labels = [2] * len(spanish_words)

# Combine datasets
all_words = english_words + german_words + spanish_words
all_labels = english_labels + german_labels + spanish_labels

# Convert words to numerical features using language-specific mappings
def create_char_mapping(language):
    if language == "english":
        alphabet = 'abcdefghijklmnopqrstuvwxyz'
    elif language == "german":
        alphabet = 'abcdefghijklmnopqrstuvwxyzäöüß'
    elif language == "spanish":
        alphabet = 'abcdefghijklmnopqrstuvwxyzáéíóúüñ'
    else:
        raise ValueError("Unsupported language")

    mapping = {}
    for idx, char in enumerate(alphabet, start=1):
        mapping[char] = idx
    return mapping

english_mapping = create_char_mapping("english")
german_mapping = create_char_mapping("german")
spanish_mapping = create_char_mapping("spanish")

def word_to_features(word, language):
    if language == 0:
        mapping = english_mapping
    elif language == 1:
        mapping = german_mapping
    elif language == 2:
        mapping = spanish_mapping
    else:
        raise ValueError("Unsupported language label")

    return [mapping.get(char.lower(), 0) for char in word]

features = []
for word, label in zip(all_words, all_labels):
    features.append(word_to_features(word, label))

# Create a DataFrame for features and labels
data = pd.DataFrame(features)
data['label'] = all_labels

# Pad sequences to ensure uniform length
max_length = data.shape[1] - 1
data = data.apply(lambda row: row.fillna(0), axis=1)

# Split data into training and testing sets
X = data.iloc[:, :-1]
y = data['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train and evaluate models
models = {
    "KNN": KNeighborsClassifier(n_neighbors=3),
    "SVM": SVC(kernel='linear', probability=True),
    "MLP": MLPClassifier(hidden_layer_sizes=(100,), max_iter=500, random_state=42)
}

results = {}
for model_name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    results[model_name] = accuracy
    print(f"{model_name} Accuracy: {accuracy * 100:.2f}%")

# Plot results
plt.barh(list(results.keys()), list(results.values()))
plt.title('Model Accuracy Comparison')
plt.xlabel('Accuracy')
plt.ylabel('Model')
plt.xticks(rotation=45)
plt.yticks(rotation=0)
plt.show()

Mounted at /content/drive
KNN Accuracy: 75.01%
